In [1]:
!git clone https://github.com/alexanderquispe/naics-github-train.git

Cloning into 'naics-github-train'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 58 (delta 20), reused 54 (delta 16), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 32.39 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd naics-github-train

/content/naics-github-train


In [4]:
!python scripts/train.py \
      --model roberta-large \
      --data data/raw/train_data_gpt_ab8_score_with_code.parquet \
      --batch-size 32 \
      --epochs 8

2026-02-07 22:12:40,602 - INFO - NumExpr defaulting to 12 threads.
2026-02-07 22:12:41,531 - INFO - TensorFlow version 2.19.0 available.
2026-02-07 22:12:41,532 - INFO - JAX version 0.7.2 available.
2026-02-07 22:12:57,999 - INFO - ============================================================
2026-02-07 22:12:57,999 - INFO - NAICS GitHub Repository Classifier - Training
2026-02-07 22:12:58,000 - INFO - ============================================================
2026-02-07 22:12:58,000 - INFO - Model: roberta-large
2026-02-07 22:12:58,000 - INFO - Data: data/raw/train_data_gpt_ab8_score_with_code.parquet
2026-02-07 22:12:58,000 - INFO - Min samples per class: 80
2026-02-07 22:12:58,000 - INFO - Epochs: 8
2026-02-07 22:12:58,000 - INFO - Batch size: 32
2026-02-07 22:12:58,000 - INFO - Learning rate: 1.5e-05
2026-02-07 22:12:58,000 - INFO - Early stopping patience: 5
2026-02-07 22:12:58,000 - INFO - Gradient checkpointing: False
2026-02-07 22:12:58,000 - INFO - Output directory: /content/

In [5]:
# 1. Push to Hugging Face Hub (private initially)
from huggingface_hub import login
login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Your model was saved here after training
model_path = "models/roberta-large-naics-classifier"

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

print(f"Model loaded with {model.config.num_labels} labels")

Loading weights:   0%|          | 0/393 [00:00<?, ?it/s]

Model loaded with 19 labels


In [7]:
repo_name = "aquiro1994/naics-github-classifier"  # Change to your username

# Push model and tokenizer
model.push_to_hub(repo_name, private=True)
tokenizer.push_to_hub(repo_name, private=True)

print(f"Model pushed to: https://huggingface.co/{repo_name}")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...akpm9j8/model.safetensors:   0%|          |  554kB / 1.42GB            

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model pushed to: https://huggingface.co/aquiro1994/naics-github-classifier


In [8]:
import json
from huggingface_hub import upload_file

# Your label mappings (from training)
label2id = model.config.label2id
id2label = model.config.id2label

# Save to file
with open("label_mappings.json", "w") as f:
    json.dump({
        "label2id": label2id,
        "id2label": id2label,
        "naics_names": {
            "11": "Agriculture",
            "21": "Mining",
            "22": "Utilities",
            "23": "Construction",
            "31-33": "Manufacturing",
            "42": "Wholesale Trade",
            "44-45": "Retail Trade",
            "48-49": "Transportation",
            "51": "Information",
            "52": "Finance",
            "53": "Real Estate",
            "54": "Professional Services",
            "56": "Administrative",
            "61": "Education",
            "62": "Health Care",
            "71": "Arts/Entertainment",
            "72": "Accommodation/Food",
            "81": "Other Services",
            "92": "Public Administration"
        }
    }, f, indent=2)

# Upload to Hub
upload_file(
    path_or_fileobj="label_mappings.json",
    path_in_repo="label_mappings.json",
    repo_id=repo_name,
    repo_type="model"
)

print("Label mappings uploaded!")

Label mappings uploaded!


In [9]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model=repo_name,
    token=True  # Uses your logged-in token
)

# Test
text = "Repository: bank-api | Description: REST API for banking transactions | README: Secure financial API"
result = classifier(text)

print(result)

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/393 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/358 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

[{'label': '52', 'score': 0.9368032217025757}]
